---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [591]:
def blight_model():
    
    # import libraries from scikit learn
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    
    #load training, test, and location data
    data_train = pd.read_csv('train.csv', encoding = "ISO-8859-1")
    data_test = pd.read_csv('test.csv')
    data_address = pd.read_csv('addresses.csv')
    data_latlong = pd.read_csv('latlons.csv')
    
    # convert data to type
    data_train['city'] = data_train['city'].astype(str)
    data_test['city'] = data_test['city'].astype(str)

    data_train['violation_street_name'] = data_train['violation_street_name'].astype(str)
    data_test['violation_street_name'] = data_test['violation_street_name'].astype(str)

    data_train['zip_code'] = data_train['zip_code'].astype(str)
    data_test['zip_code'] = data_test['zip_code'].astype(str)

    data_train['violator_name'] = data_train['violator_name'].astype(str)
    data_test['violator_name'] = data_test['violator_name'].astype(str)


    data_train['violation_zip_code'] = data_train['violation_zip_code'].astype(str)
    data_test['violation_zip_code'] = data_test['violation_zip_code'].astype(str)
    
    
    #create boolean features from floats
    data_test['late_fee_bool'] = np.where(data_test['late_fee']>0,1,0)
    data_train['late_fee_bool'] = np.where(data_train['late_fee']>0,1,0)

    data_test['discount_amount_bool'] = np.where(data_test['discount_amount']>0,1,0)
    data_train['discount_amount_bool'] = np.where(data_train['discount_amount']>0,1,0)

    data_test['street_name_bool'] = np.where(
    data_test['mailing_address_str_name'] == data_test['violation_street_name']
    ,1,0)

    data_train['street_name_bool'] = np.where(
    data_train['mailing_address_str_name'] == data_train['violation_street_name']
    ,1,0)

    data_test['zip_code_bool'] = np.where(
    data_test['violation_zip_code'] == data_test['zip_code']
    ,1,0)

    data_train['zip_code_bool'] = np.where(
    data_train['violation_zip_code'] == data_train['zip_code']
    ,1,0)

    
    #set features to use in model
    starting_features = ['agency_name', 'inspector_name', 'violation_code', 'late_fee_bool', 'discount_amount_bool',
                         'judgment_amount', 'zip_code', 'disposition', 'violation_zip_code', 'zip_code_bool',
                         'late_fee','fine_amount','admin_fee','state_fee']
    
    #drop NAN from dataset for compliance
    data_train = data_train[np.isfinite(data_train['compliance'])]
    
    #Split training data into target y and feature x datasets. Split into train and test groups.
    data_train_y = data_train['compliance']
    data_train_X = data_train[starting_features]
    X_train, X_test, y_train, y_test = train_test_split(data_train_X, data_train_y, random_state=0)
    
    #encode training and test data
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder() # encoder to do label encoder
    X_train_t = X_train.apply(lambda x: le.fit_transform(x))
    X_test_t = X_test.apply(lambda x: le.fit_transform(x))
    
    #fit model
    rf_model = RandomForestClassifier(n_estimators=100, max_depth=8, random_state = 0)
    rf_model = rf_model.fit(X_train_t, y_train)
    
    #prepare test data to be scored
    data_test = data_test.set_index('ticket_id')
    data_test_features = data_test[starting_features]
    data_test_features_t = data_test_features.apply(lambda x: le.fit_transform(x))
    
    #predict probabilities for test data
    predictions = rf_model.predict_proba(data_test_features_t)
    
    #get probabilities for positive class
    prediction_pos = predictions[:,1]
    
    #add data to the dataframe
    data_test_scored = data_test_features
    data_test_scored ['predictions'] = prediction_pos
    
    #produce series answer
    answer = data_test_scored['predictions']
    
    return answer

print(blight_model())

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3220: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


ticket_id
284932    0.145435
285362    0.053871
285361    0.135225
285338    0.163994
285346    0.154824
285345    0.171171
285347    0.185702
285342    0.784041
285530    0.053442
284989    0.108673
285344    0.186340
285343    0.081581
285340    0.055599
285341    0.186611
285349    0.142690
285348    0.149414
284991    0.103724
285532    0.114620
285406    0.112528
285001    0.128658
285006    0.072013
285405    0.039612
285337    0.108210
285496    0.160152
285497    0.149339
285378    0.062249
285589    0.107738
285585    0.150361
285501    0.125006
285581    0.046871
            ...   
376367    0.059190
376366    0.104801
376362    0.104801
376363    0.119277
376365    0.059190
376364    0.104801
376228    0.133548
376265    0.112366
376286    0.748704
376320    0.105810
376314    0.105235
376327    0.738766
376385    0.749388
376435    0.737936
376370    0.758613
376434    0.182424
376459    0.212689
376478    0.081376
376473    0.106687
376484    0.112916
376482    0.103316
37

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [82]:
#Notes for next steps
# 1. Convert everything to category and try more features(https://stackoverflow.com/questions/46406720/labelencoder-typeerror-not-supported-between-instances-of-float-and-str?rq=1)
# 2. Convert "late fee" to Binary Variable
# 3. Determine whether floats / integers should be left as such or if it doesn't matter
# 4. Try to do binning for floats variables (e.g. judgement amount)
# 5. Map Address to Lat Long
# 6. Do clean-up on names to make sure they're consistent
# 7. Check if mailing address is the same as street address

In [389]:
import pandas as pd
import numpy as np

data_train = pd.read_csv('train.csv', encoding = "ISO-8859-1")
data_test = pd.read_csv('test.csv',encoding = "ISO-8859-1")
data_address = pd.read_csv('addresses.csv')
data_latlong = pd.read_csv('latlons.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [390]:
# convert data to type
data_train['city'] = data_train['city'].astype(str)
data_test['city'] = data_test['city'].astype(str)

data_train['violation_street_name'] = data_train['violation_street_name'].astype(str)
data_test['violation_street_name'] = data_test['violation_street_name'].astype(str)

data_train['zip_code'] = data_train['zip_code'].astype(str)
data_test['zip_code'] = data_test['zip_code'].astype(str)

data_train['violator_name'] = data_train['violator_name'].astype(str)
data_test['violator_name'] = data_test['violator_name'].astype(str)


data_train['violation_zip_code'] = data_train['violation_zip_code'].astype(str)
data_test['violation_zip_code'] = data_test['violation_zip_code'].astype(str)


In [391]:
# add new features as new columns in dataframe

In [392]:
data_test.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status'],
      dtype='object')

In [393]:
#data_test['discount_amount'].unique()
#data_test['late_fee'].where(data_test['late_fee']==0)

data_test['late_fee_bool'] = np.where(data_test['late_fee']>0,1,0)
data_train['late_fee_bool'] = np.where(data_train['late_fee']>0,1,0)

data_test['discount_amount_bool'] = np.where(data_test['discount_amount']>0,1,0)
data_train['discount_amount_bool'] = np.where(data_train['discount_amount']>0,1,0)

data_test['street_name_bool'] = np.where(
    data_test['mailing_address_str_name'] == data_test['violation_street_name']
    ,1,0)

data_train['street_name_bool'] = np.where(
    data_train['mailing_address_str_name'] == data_train['violation_street_name']
    ,1,0)

data_test['zip_code_bool'] = np.where(
    data_test['violation_zip_code'] == data_test['zip_code']
    ,1,0)

data_train['zip_code_bool'] = np.where(
    data_train['violation_zip_code'] == data_train['zip_code']
    ,1,0)


count=0

for i in range(0,len(data_test['mailing_address_str_name'])):
    if data_test['violation_zip_code'][i] == data_test['zip_code'][i]:
        count+=1
print(count)

#print(data_test['mailing_address_str_name'][1],data_test['violation_street_name'][1])

#(data_test['mailing_address_str_name'][i] == data_test['violation_street_name'][i]) &

8609


In [464]:
#import matplotlib.pyplot as plt

#data_test['judgment_amount'].unique

#plt.hist(data_test['judgment_amount'],30, range=(0,2000))

#plt.figure()
#plt.scatter(data_test['judgment_amount'].index,data_test['judgment_amount'])

In [592]:
starting_features = ['agency_name', 'inspector_name', 'violation_code', 'late_fee_bool', 'discount_amount_bool',
            'judgment_amount', 'zip_code', 'disposition', 'violation_zip_code',
                    'late_fee','fine_amount','admin_fee','state_fee']

        
#starting_features=[ 'late_fee','fine_amount','admin_fee','state_fee','judgment_amount','discount_amount']
#categorical_features = ['agency_name', 'inspector_name','violation_description']
#type(starting_features)

In [593]:
#drop NAN from dataset
data_train = data_train[np.isfinite(data_train['compliance'])]

In [594]:
#training data split
data_train_y = data_train['compliance']
data_train_X = data_train[starting_features]
X_train, X_test, y_train, y_test = train_test_split(data_train_X, data_train_y, random_state=0, test_size=0.2)

In [595]:
print(len(X_train),len(X_test),len(y_train),len(y_test))

127904 31976 127904 31976


In [596]:
#agency_name_values = X_train['agency_name'].values
#inspector_name_values = X_train['inspector_name'].values
#print(X_train.columns)
#print(agency_name_values)
#print(inspector_name_values)

In [597]:
for i in X_train.columns:
    print(i, len(X_train.columns))

agency_name 13
inspector_name 13
violation_code 13
late_fee_bool 13
discount_amount_bool 13
judgment_amount 13
zip_code 13
disposition 13
violation_zip_code 13
late_fee 13
fine_amount 13
admin_fee 13
state_fee 13


In [598]:
from sklearn.preprocessing import LabelEncoder

#encode training data
le = LabelEncoder() # encoder to do label encoder
X_train_t = X_train.apply(lambda x: le.fit_transform(x))
X_test_t = X_test.apply(lambda x: le.fit_transform(x))

In [599]:
X_test_t.head()

,agency_name,inspector_name,violation_code,late_fee_bool,discount_amount_bool,judgment_amount,zip_code,disposition,violation_zip_code,late_fee,fine_amount,admin_fee,state_fee
248437,0,59,101,1,0,22,749,2,0,12,13,0,0
113965,0,30,117,1,0,14,758,2,0,7,8,0,0
150096,0,49,69,0,0,1,793,1,0,0,1,0,0
84976,0,119,117,1,0,14,727,2,0,7,8,0,0
3805,0,113,97,1,0,14,741,2,0,7,8,0,0


In [610]:
#fit model
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state = 0)
rf_model = rf_model.fit(X_train_t, y_train)

In [611]:
print(rf_model.score(X_train_t,y_train))
print(rf_model.score(X_test_t,y_test))

0.9442316112084063
0.9310733049787341


In [612]:
#encode test data
data_test_features = data_test[starting_features]
data_test_features_t = data_test_features.apply(lambda x: le.fit_transform(x))

In [613]:
predictions = rf_model.predict_proba(data_test_features_t)
print(type(predictions),len(predictions))

<class 'numpy.ndarray'> 61001


In [614]:
prediction_pos = predictions[:,1]

In [615]:
data_test_scored = data_test_features
data_test_scored ['predictions'] = prediction_pos

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [616]:
answer = data_test_scored['predictions']

In [617]:
pd.unique(answer)

array([0.10490992, 0.06351879, 0.12732046, ..., 0.06359757, 0.08157437,
       0.04687968])

In [618]:
from sklearn.metrics import roc_curve, auc
y_predict_rf = rf_model.predict_proba(X_test_t)[:,1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_predict_rf)
roc_auc_rf = auc(fpr_rf, tpr_rf)

In [619]:
roc_auc_rf

0.7837596725751897